In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # Or use Ollama/Mistral
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="granite3.3:8b")  # Or "llama2", "codellama", etc.

# Instead of llm.name, use:
print(f"Model: {llm.model}")  # This will show "granite3.3:8b"
# Or to see all available attributes:
print(f"LLM type: {type(llm).__name__}")  # This will show "OllamaLLM"


# Pattern Scanner Agent
pattern_prompt = PromptTemplate(
    input_variables=["code"],
    template="You are a code reviewer. Find and name any anti-patterns in this Java code:\n\n{code}"
)
PatternScannerAgent = LLMChain(llm=llm, prompt=pattern_prompt)

# Refactoring Agent
refactor_prompt = PromptTemplate(
    input_variables=["code", "issues"],
    template="Refactor the Java code below by addressing the following anti-patterns: {issues}\n\nCode:\n{code}"
)
RefactoringAgent = LLMChain(llm=llm, prompt=refactor_prompt)

Model: granite3.3:8b
LLM type: OllamaLLM


/tmp/ipykernel_2177642/2156565391.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  PatternScannerAgent = LLMChain(llm=llm, prompt=pattern_prompt)


In [2]:
from langgraph.graph import StateGraph
from typing import TypedDict

# Shared context object (like MCP envelope)
class AgentState(TypedDict): 
    code: str  # The original code to be scanned and refactored
    issues: str  # Issues found by the pattern scanner
    refactored: str  # Refactored code after addressing issues
    # def __init__(self, code: str, issues: str = "", refactored: str = ""):
    #     self.code = code
    #     self.issues = issues
    #     self.refactored = refactored

# Pattern Scanner Node

def scan_for_patterns(state: AgentState):
    print("🔍 Scanning for anti-patterns...")
    state["issues"] = PatternScannerAgent.run(code=state["code"])
    return state

# Refactoring Agent Node

def refactor_code(state: AgentState):
    print("🛠️ Refactoring code...")
    state["refactored"] = RefactoringAgent.run(code=state["code"], issues=state["issues"])
    return state

# Terminal Display Node

def display_results(state: AgentState):
    print("✅ Done.\nIssues Found:\n", state["issues"])
    print("Refactored Code:\n", state["refactored"])
    return state

In [3]:
graph = StateGraph(AgentState)
graph.add_node("scan_for_patterns", scan_for_patterns)
graph.add_node("refactor_code", refactor_code)
graph.add_node("display_results", display_results)

# Define flow: Scanner → Refactorer → Display
graph.set_entry_point("scan_for_patterns")
graph.add_edge("scan_for_patterns", "refactor_code")
graph.add_edge("refactor_code", "display_results")

# Compile the FSM graph
agent_pipeline = graph.compile()

In [5]:
# Sample legacy code input
legacy_code = """
public class ApplicationManager {
    private List<String> users = new ArrayList<>();
    private List<String> logs = new ArrayList<>();
 
    public void addUser(String user) {
        users.add(user);
        logs.add("User added: " + user);
    }
 
    public void removeUser(String user) {
        users.remove(user);
        logs.add("User removed: " + user);
    }
 
    public void logEvent(String event) {
        logs.add(event);
    }
 
    public void printReport() {
        System.out.println("=== Users ===");
        for (String user : users) {
            System.out.println(user);
        }
 
        System.out.println("=== Logs ===");
        for (String log : logs) {
            System.out.println(log);
        }
    }
 
    public void backupData() {
        // Placeholder: pretend this backs up all data
        System.out.println("Backing up users and logs...");
    }
}
"""

initial_state = AgentState(code=legacy_code)
final_state = agent_pipeline.invoke(initial_state)

🔍 Scanning for anti-patterns...
🛠️ Refactoring code...
✅ Done.
Issues Found:
 1. **God Class Anti-pattern**: The `ApplicationManager` class is handling multiple responsibilities, including managing users, logging events, printing reports, and backing up data. This violates the Single Responsibility Principle (SRP), which states that a class should have only one reason to change.

2. **Data Duplication**: The `users` list stores actual user data, while the `logs` list records events related to those users. There is no clear separation of concerns between data storage and event logging. This could lead to inconsistencies if the formats or structures of these lists diverge.

3. **Lack of Error Handling**: The methods like `addUser`, `removeUser`, and `backupData` do not handle potential exceptions, such as adding a null user or encountering issues while backing up data. Proper exception handling should be included to make the code more robust.

4. **No Encapsulation**: Both `users` and `l